# Combination of all important functions
This is a summary of the important functions for constructing a network and solving it for the currents.   
First off there is the Construct function which takes the paths to the input data and builds a corresponding Networkx-Graph from it.  
Then there ist the Solve class which takes a network as input and can then construct the matrices and vectors needed to solve for the currents.   
In the end there is a snippet attached which can be used to orient the edges/current-directions. It takes a incidence-like matrix as input and changes the signs according to the potential landscape.

*Stand 10.07.23*

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy import sparse
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def Construct(Coo, Typ, Ener):
    #Read coordinates from file
    coords = list(map(tuple, np.genfromtxt(Coo, delimiter=" ")))
    #Build Graph
    F=nx.Graph()
    F.add_nodes_from(coords)
    #Read Attributes
    types = np.genfromtxt(Typ, delimiter=" ")
    energies = np.genfromtxt(Ener, delimiter=" ")
    #Set Attributes
    i=0
    for u in F.nodes():
        F.nodes[u]["pos"]=u
        F.nodes[u]["type"]=types[i]
        F.nodes[u]["potential"]=energies[i]
        i+=1  
    #Construct Edges
    F.add_edges_from(nx.geometric_edges(F, 1))
    
    ## Inizializing with random values until access to resistivity values ##
    R_0=1
    sigmaR=0.05
    for (u, v) in F.edges(): #die beiden Endpunkte u und v der Kante
        F.edges[u,v]['weight'] = random.normalvariate(R_0, sigmaR)
        
    ## These Edges should maybe be oriented by using some variation of the code in the end ##
    
    return F


#Gitter=Construct("coord_0.dat", "mol_types_0.dat", "site_energies_0.dat")

In [3]:
class Solve:
    def __init__(self, X):
        self.G=X
        self.N=self.G.order()
        self.nR=self.G.number_of_edges()
        
    def conductances(self):
        #Extract the values of the resistors from the graph and build a nR x nR matrix
        mat=sparse.spdiags(1/np.asarray(list((nx.get_edge_attributes(self.G, "weight").values()))), 0, self.nR, self.nR)
        return sparse.csc_matrix(mat)
    
    def incidence(self):
        #Builds the incidence matrix from the graph
        mat= np.transpose(nx.incidence_matrix(self.G, oriented=1)) #Beachte Transpose damit die Dimensionen der networkx funktion zum paper passen
        
        
        ## Siehe Codeschnipsel im ersten Beispiel falls die Orientierung ungünstig ist ##
        
        
        return mat
    
    def voltages(self):
        #Get the potential values from the nodes and build a vector
        vec=np.array(list(nx.get_node_attributes(self.G, "potential").values()))
        return vec

    
    def currents(self):
        #Combines the other functions to get the currents trough the resistors
        return - (self.conductances() @ self.incidence()) @ self.voltages()

    
#Solve(Gitter).currents()   

In [4]:
def Orientation(G):
    #Code Snippet that could be used for orienting the edges
    Mat=G.incidence()
    V=G.voltages()
    for x in range(np.shape(Mat)[0]):
        Y=[]
        for y in range(np.shape(Mat)[1]):
            if Mat[x,y] !=0:
                Y.append(y)
        Mat[x,Y[0]]=-np.sign(V[Y[0]]-V[Y[1]])
        Mat[x,Y[1]]=-Mat[x,Y[0]]
    return Mat

In [5]:
Gitter=Construct("coord_0.dat", "mol_types_0.dat", "site_energies_0.dat")
Setup=Solve(Gitter)
Setup.currents()

array([-0.04893047, -0.24495738,  0.19003275, -0.01435548,  0.11944639,
        0.064607  , -0.07259912, -0.16781496, -0.05076602,  0.28155887,
       -0.15286428, -0.07388206,  0.1264492 , -0.04126597, -0.05593821,
        0.04227985,  0.14362606, -0.22407117,  0.08438378, -0.07291825,
        0.12052241, -0.20621719,  0.0443407 ,  0.1363038 , -0.11045878,
        0.26027446, -0.14385933,  0.0334608 , -0.00070696, -0.24150709,
        0.13663153, -0.08674355,  0.13821833, -0.07815807,  0.00488968,
        0.01054313,  0.19764378, -0.19798458,  0.11573613, -0.05391098,
        0.00333507,  0.11879777, -0.25610251,  0.19261454,  0.05559996,
       -0.09729242, -0.01911469])